In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [26]:

urls = []
price = []
area = []
area_unit = []
area_type = []
furnishing = []
contact = []

for i in range(1,350):
    page_url = f"https://www.squareyards.com/rent/apartments-for-rent-in-delhi?page={i}"

    response = requests.get(page_url)
    soup = BeautifulSoup(response.text, "html.parser")

    scripts = soup.find_all("script", type = "application/ld+json")
    for script in scripts:
        dic = json.loads(script.string)
        if dic["@type"] == "Apartment":
            urls.append(dic["url"])

    listing_prices = soup.find_all("p", class_ = "listing-price")
    for listing in listing_prices:
        listing_price = listing.find("strong").text
        price.append(listing_price)

    listing_infos = soup.find_all("ul", class_ = "listing-information")
    for listing in listing_infos:
        listing_area = float(listing.find_all("span")[1].contents[0].strip().split(" ")[0])
        area.append(listing_area)
        listing_area_unit = listing.find_all("span")[1].contents[0].strip().split(" ")[1]
        area_unit.append(listing_area_unit)

        listing_area_type = listing.find_all("span")[1].find("b").text[1:-1]
        area_type.append(listing_area_type)

        furnishing_cond = listing.find_all("span")[0].text
        furnishing.append(furnishing_cond)

    contact_infos = soup.find_all('div', class_ = "btn btn-primary openCommonLeadForm")
    for contact_info in contact_infos:
        contact.append(contact_info.find("span").text.strip())

In [28]:
zone = []
n_room = []
n_bathroom = []
facing = []
floor = []
total_floors = []
power_backup = []
available_from = []

for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    table = soup.find("table", class_ = "propertyInfoTable")

    info_dict = {}
    for item in table.find_all("tr"):
        key = item.find("td").contents[0]
        value = item.find("strong").text.strip()

        info_dict[key] = value
    
    zone.append(info_dict.get("Micro market"))
    n_room.append(info_dict.get("Number of Rooms"))
    n_bathroom.append(info_dict.get("Number of Bathroom"))
    facing.append(info_dict.get("Facing"))
    floor.append(info_dict.get("Floor Number"))
    total_floors.append(info_dict.get("Total Floor Count"))
    power_backup.append(info_dict.get("Power Back-up"))
    available_from.append(info_dict.get("Available From"))


In [29]:
df = pd.DataFrame(
    {
        "zone": zone,
        "room": n_room,
        "bathroom": n_bathroom,
        "floor": floor,
        "t_floors": total_floors,
        "area": area,
        "area_unit": area_unit,
        "area_type": area_type,
        "furnishing": furnishing,
        "facing": facing,
        "power_backup": power_backup,
        "available_from": available_from,
        "contact": contact,
        "rent": price
    },
)
df

,zone,room,bathroom,floor,t_floors,area,area_unit,area_type,furnishing,facing,power_backup,available_from,contact,rent
0,North Delhi,3,2,3,None,1400.0,Sq.Ft.,Built-up Area,Furnished,East,Available,Immediately,Contact Agent,"₹ 52,000"
1,South Delhi,2,2,02nd floor,04,900.0,Sq.Ft.,Built-up Area,Semi-Furnished,South,No Back-up,Immediately,Contact Agent,"₹ 50,000"
2,South Delhi,3,3,Second Floor With lift,4rd Floor,1800.0,Sq.Ft.,Built-up Area,Unfurnished,East,Available,Immediately,Contact Agent,"₹ 60,000"
3,Dwarka,3,3,5,10,1700.0,Sq.Ft.,Carpet Area,Semi-Furnished,North East,Available,None,Contact Agent,"₹ 70,000"
4,North Delhi,3,2,3,4,1300.0,Sq.Ft.,Built-up Area,Furnished,East,Available,Later,Contact Agent,"₹ 54,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5483,South Delhi,3,3,FIRST FLOOR,4TH FKLOOR,2000.0,Sq.Ft.,Built-up Area,Furnished,East,None,Immediately,Contact Agent,₹ 1.25 L
5484,South Delhi,1,1,3,5,450.0,Sq.Ft.,Built-up Area,Semi-Furnished,East,No Back-up,Immediately,Contact Agent,Price on Request
5485,South Delhi,1 Rk,1,ground floor,4,300.0,Sq.Ft.,Built-up Area,Semi-Furnished,West,No Back-up,Immediately,Contact Agent,"₹ 16,000"
5486,South Delhi,4,4,SECOND FLOOR,4TH FLOOR,3200.0,Sq.Ft.,Built-up Area,Semi-Furnished,None,None,Immediately,Contact Agent,₹ 2.1 L


In [ ]:
df.to_csv("rent.csv")